<h3>Data Aquisition w/o Face and Normalised

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp
import sklearn
import tensorflow

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM ,Dense
from tensorflow.keras.callbacks import TensorBoard

In [3]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [4]:
mp_holistic = mp.solutions.holistic #makes detections
mp_drawing = mp.solutions.drawing_utils #draws detections

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False   #image not writeable
    results = model.process(image)  #make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                                                                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                                                                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                                                                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    

In [7]:
def normalize_keypoints(keypoints, center_keypoint, reference_distance):
    # Reshape the keypoints into (x, y, z) coordinates
    keypoints = keypoints.reshape(-1, 3)
    
    # Subtract center keypoint to get relative coordinates
    relative_keypoints = keypoints - center_keypoint
    
    # If reference distance is provided, scale the keypoints
    relative_keypoints = relative_keypoints / reference_distance
    #relative_keypoints = keypoints / reference_distance
    

    return relative_keypoints.flatten()

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    pose=pose[:69]
    
    return np.concatenate([pose,lh,rh])

In [10]:
Data_Path = os.path.join("ISL_Data_Prashant")

In [276]:
actions = np.array(["Not"])
no_sequences = 50  #60 videos
sequence_length = 30    #30 frames each

In [277]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(Data_Path, action, str(sequence)))
        except:
            pass

In [278]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        
        for sequence in range(20,50):
            
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                              
                draw_landmarks(image, results)
                            
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)               
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(Data_Path, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [192]:
cap.release()
cv2.destroyAllWindows()

In [10]:
actions = np.array(["Father", "What","Why"])

In [11]:
label_map = {label:num for num , label in enumerate(actions)}

In [12]:
label_map

{'Father': 0, 'What': 1, 'Why': 2}

In [17]:
""" np.array(sequences).shape   #2 videos * 2 classes , 30 frames each , 225 keypoints in each frame """

' np.array(sequences).shape   #2 videos * 2 classes , 30 frames each , 225 keypoints in each frame '

In [18]:
""" np.array(labels).shape """

' np.array(labels).shape '

In [9]:
import numpy as np
import mediapipe as mp

def reconstruct_keypoints(flattened_array):
    # Number of landmarks and features
    num_pose_landmarks = 23
    
    num_hand_landmarks = 21
    num_pose_features = 3
    num_hand_features = 3
   
    
    # Determine the indices for slicing
    pose_end = num_pose_landmarks * num_pose_features
    
    lh_end = pose_end + num_hand_landmarks * num_hand_features
    rh_end = lh_end + num_hand_landmarks * num_hand_features

    # Extract landmarks from flattened array
    pose_flat = flattened_array[:pose_end]
    
    lh_flat = flattened_array[pose_end:lh_end]
    rh_flat = flattened_array[lh_end:rh_end]

    # Reshape back to original format
    pose = np.reshape(pose_flat, (num_pose_landmarks, num_pose_features))
    lh = np.reshape(lh_flat, (num_hand_landmarks, num_hand_features))
    rh = np.reshape(rh_flat, (num_hand_landmarks, num_hand_features))

    # Convert to list of tuples
    pose_landmarks = [{'x': pose[i, 0], 'y': pose[i, 1], 'z': pose[i, 2], } for i in range(num_pose_landmarks)]
    lh_landmarks = [{'x': lh[i, 0], 'y': lh[i, 1], 'z': lh[i, 2]} for i in range(num_hand_landmarks)]
    rh_landmarks = [{'x': rh[i, 0], 'y': rh[i, 1], 'z': rh[i, 2]} for i in range(num_hand_landmarks)]

    return pose_landmarks, lh_landmarks, rh_landmarks


In [10]:
def draw_landmarks_on_black(image_shape, pose_landmarks, lh_landmarks, rh_landmarks):
    """Draw landmarks on a black background."""
    # Create a black image
    image = np.zeros(image_shape, dtype=np.uint8)
    
    # Helper function to draw landmarks
    def draw_landmark_list(landmarks, color):
        for landmark in landmarks:
            x = int(landmark['x'] * image_shape[1])
            y = int(landmark['y'] * image_shape[0])
            cv2.circle(image, (x, y), 3, color, -1)
    
    
    
    # Draw pose landmarks
    if pose_landmarks:
        draw_landmark_list(pose_landmarks, (255, 0, 0))  # Red for pose
    
    # Draw left hand landmarks
    if lh_landmarks:
        draw_landmark_list(lh_landmarks, (255, 255, 0))  # Yellow for left hand
    
    # Draw right hand landmarks
    if rh_landmarks:
        draw_landmark_list(rh_landmarks, (0, 255, 255))  # Cyan for right hand
    
    return image



In [19]:
image_shape = (500, 500, 3)
for j in range(40,60):
    for i in range(30):
        
        path = f"ISL_Data_Shubham/I/{j}"
        #path = f"test video/{j}"
        frame = np.load(path+f"/{i}.npy")
        
        center_keypoint = frame[0:3]  # Nose keypoint (x, y, z)
    
        left_shoulder = frame[11*3:11*3+3]
        right_shoulder = frame[12*3:12*3+3]
        reference_distance = np.linalg.norm(left_shoulder - right_shoulder)
        if not reference_distance:
            reference_distance=1
        # Normalize pose, left hand, and right hand keypoints with respect to the common keypoint
        #frame = normalize_keypoints(frame, center_keypoint, reference_distance)

        pose_landmarks, lh_landmarks, rh_landmarks = reconstruct_keypoints(frame)

        plots = draw_landmarks_on_black(image_shape, pose_landmarks, lh_landmarks, rh_landmarks)

        cv2.imshow('Landmarks', plots)
        cv2.waitKey(20)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

In [17]:
cv2.destroyAllWindows()